In [ ]:
%cd ..
!rm -rf epigraphnet

: 

In [1]:
!git clone https://github.com/0nur0duncu/epigraphnet.git

Cloning into 'epigraphnet'...
remote: Enumerating objects: 586, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (566/566), done.
remote: Total 586 (delta 35), reused 570 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (586/586), 2.76 MiB | 19.66 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [16]:
!git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 4.93 KiB | 1.64 MiB/s, done.
From https://github.com/0nur0duncu/epigraphnet
   566bee0..ddcd16e  main       -> origin/main
Updating 566bee0..ddcd16e
Fast-forward
 main.ipynb            | 526 ++++++++++++++++++++++++++++++++------------------
 models/epigraphnet.py |  59 +-----
 models/gcn_module.py  |  93 ++++-----
 3 files changed, 392 insertions(+), 286 deletions(-)


In [2]:
%cd epigraphnet

/content/epigraphnet


## 2. Dizin Değiştirme

In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.1 MB/s eta 0:00:0000:01


## 3. Bağımlılıkları Yükleme

In [4]:
!python data/download_bonn.py --force

Mevcut veri siliniyor: data/bonn
Bonn Epileptik EEG Veri Kümesi İndirme
Kaynak: GitHub - RYH2077/EEG-Epilepsy-Datasets

Veri seti indiriliyor...
  İlerleme: 100.0%
  ✓ İndirme tamamlandı!

Çıkarılıyor: data/bonn/bonn_dataset.zip
  ✓ 500 dosya çıkarıldı!

✓ Toplam 500 dosya başarıyla indirildi!
  Konum: /content/epigraphnet/data/bonn
✓ Veri kümesi doğrulandı!


In [18]:
# Eğitimi başlat
# UYARI: İlk çalıştırmada Bonn veri kümesi otomatik indirilecektir
!python train.py --config config/config.yaml

Cihaz: cuda

Veri kümesi bulunamadı veya eksik: data/bonn
Veri kümesi hazırlanıyor...
✓ Veri zaten mevcut: data/bonn (500 dosya)
Veri yüklendi:
  Eğitim: 400 örnek
  Validasyon: 50 örnek
  Test: 50 örnek
Model oluşturuldu: EpiGraphNet

Epoch 1/50
Train: loss: 1.95 (best: 1.95) | accuracy: 18.23 (best: 18.23) | precision: 14.30 (best: 14.30) | recall: 18.36 (best: 18.36) | f1: 9.42 (best: 9.42)
Val:   loss: 1.87 (best: 1.87) | accuracy: 20.00 (best: 20.00) | precision: 4.00 (best: 4.00) | recall: 20.00 (best: 20.00) | f1: 6.67 (best: 6.67)
✓ En iyi model kaydedildi: checkpoints/best_model.pt

Epoch 2/50
Train: loss: 1.87 (best: 1.87) | accuracy: 19.53 (best: 19.53) | precision: 8.38 (best: 14.30) | recall: 19.40 (best: 19.40) | f1: 11.12 (best: 11.12)
Val:   loss: 1.79 (best: 1.79) | accuracy: 20.00 (best: 20.00) | precision: 4.00 (best: 4.00) | recall: 20.00 (best: 20.00) | f1: 6.67 (best: 6.67)
✓ En iyi model kaydedildi: checkpoints/best_model.pt

Epoch 3/50
Train: loss: 1.73 (best: 1

In [17]:
!rm -rf checkpoints/

## 4. Model Eğitimi

**Makaledeki Tablo I parametreleriyle eğitim:**
- Dataset: Bonn EEG (500 örnek, 5 sınıf)
- Batch Size: 64
- Learning Rate: 0.001
- Epochs: 50 (Early Stopping: Patience=10)
- Optimizer: Adam (weight_decay=0.0005)

**UYARI:** 
- ⏱️ Eğitim süresi: GPU'da ~10-30 dakika, CPU'da 1-2 saat
- 💾 İlk çalıştırmada Bonn veri kümesi otomatik indirilir (~15MB)
- 📊 En iyi model `checkpoints/best_model.pt` olarak kaydedilir

**Beklenen Davranış:**
- Her epoch'ta Train ve Val metrikleri görüntülenir
- Loss azalırken Accuracy artmalıdır
- Early stopping devreye girebilir (validation loss düzelmezse)

In [15]:
!!python evaluate.py --num-runs 1

['Cihaz: cuda',
 '',
 '==================================================',
 'Çalıştırma 1/1',
 '==================================================',
 'Veri yüklendi:',
 '  Eğitim: 400 örnek',
 '  Validasyon: 50 örnek',
 '  Test: 50 örnek',
 '',
 'Testing:   0% 0/1 [00:00<?, ?it/s]',
 'Testing: 100% 1/1 [00:00<00:00,  1.89it/s]',
 'Testing: 100% 1/1 [00:00<00:00,  1.89it/s]',
 'Doğruluk: 16.00%',
 'Duyarlılık: 16.00%',
 'Kesinlik: 3.33%',
 'F1: 5.52%',
 '',
 '============================================================',
 'SONUÇ ÖZETİ',
 '============================================================',
 'Model: EpiGraphNet_DE (a=50)',
 '------------------------------------------------------------',
 'Metrik          Ortalama        Std            ',
 '---------------------------------------------',
 'Accuracy           16.00%      ±  0.00%',
 'Recall             16.00%      ±  0.00%',
 'Precision           3.33%      ±  0.00%',
 'F1                  5.52%      ±  0.00%',
 '==============

In [19]:
# Makaledeki gibi 5 kez çalıştırıp ortalama ve standart sapma hesapla
!python evaluate.py --checkpoint checkpoints/best_model.pt --num-runs 5

Cihaz: cuda

Çalıştırma 1/5
Veri yüklendi:
  Eğitim: 400 örnek
  Validasyon: 50 örnek
  Test: 50 örnek
Traceback (most recent call last):
  File "/content/epigraphnet/evaluate.py", line 216, in <module>
    main()
  File "/content/epigraphnet/evaluate.py", line 205, in main
    summary = run_multiple_evaluations(
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/epigraphnet/evaluate.py", line 118, in run_multiple_evaluations
    model.load_state_dict(checkpoint["model_state_dict"])
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 2629, in load_state_dict
    raise RuntimeError(
RuntimeError: Error(s) in loading state_dict for EpiGraphNet:
	Unexpected key(s) in state_dict: "gcn.gcn_layers.0.conv.linear.weight", "gcn.gcn_layers.0.conv.linear.bias", "gcn.gcn_layers.1.conv.linear.weight", "gcn.gcn_layers.1.conv.linear.bias", "gcn.gcn_layers.2.conv.linear.weight", "gcn.gcn_layers.2.conv.linear.bias", "gcn.fc.weight", "gcn.fc.bias", "cnn_lstm.cnn.fc.we

## 5.1 Stratified K-Fold Cross-Validation (Düzeltilmiş)

**Küçük veri setlerinde (500 örnek) sorunlar:**
- ❌ Validation split (50 örnek) çok gürültülü → early stopping yanlış tetiklenir
- ❌ 350 eğitim örneği yetersiz → model iyi öğrenemiyor

**Çözüm - Düzeltilmiş K-Fold:**
- ✅ **Validation split YOK** - Tüm 400 örnek eğitim için kullanılır
- ✅ **Sabit 100 epoch** - Early stopping kaldırıldı
- ✅ **Cosine Annealing LR** - Daha iyi yakınsama
- ✅ **Best train accuracy modeli** seçilir

**Avantajları:**
- Daha fazla eğitim verisi (400 vs 350)
- Kararlı eğitim (gürültülü validation yok)
- Tam yakınsama (100 epoch)

**⚠️ UYARI:** Her fold 100 epoch. Toplam süre: ~2-3 saat (GPU)

In [ ]:
# Stratified 5-Fold Cross-Validation (Düzeltilmiş Versiyon)
# - Validation split YOK (küçük veri setlerinde gürültülü)
# - Tüm 400 train örneği kullanılır
# - 100 epoch (daha iyi yakınsama için)
# - Cosine annealing LR scheduler
!python evaluate_kfold.py --k 5 --epochs 100 --seed 42

### ⚠️ DURUM: Checkpoint Eski Mimariyle Eğitilmiş

**Checkpoint Bilgisi:**
- Epoch: 17
- Val Accuracy: 70%
- **Sorun:** Eski model mimarisi ile kaydedilmiş

**Hata:**
```
RuntimeError: Unexpected key(s) in state_dict
```

**Sebep:** Checkpoint'teki katman isimleri (gcn.gcn_layers.X.conv.lin_rel) yeni kod ile uyumsuz

---

## ✅ ÇÖZÜM: YENİ MİMARİYLE EĞİT

### 🔴 1. CHECKPOINT'İ SİL (Hücre 14 - Hemen aşağıda)
Eski checkpoint'i silmek için aşağıdaki hücreyi çalıştırın ⬇️

### 🔵 2. YENİDEN EĞİT (Hücre 8)
Modeli yeni mimariyle baştan eğitin (Scroll up → Hücre 8)
```python
!python train.py --config config/config.yaml
```
⏱️ **Süre:** 10-30 dakika (GPU), 1-2 saat (CPU)  
📊 **Hedef:** Val Accuracy > 80% (şu an eski checkpoint 70%)

### 🟢 3. DEĞERLENDİR (Hücre 12)
Eğitim bittikten sonra bu hücreyi (Hücre 12) tekrar çalıştırın
```python
!python evaluate.py --checkpoint checkpoints/best_model.pt --num-runs 5
```
🎯 **Beklenen:** Accuracy 90-95%+

In [ ]:
# 🔴 ADI M 1: ESKİ CHECKPOINT'İ SİL
import os
import shutil

checkpoint_dir = "checkpoints"

print("=" * 60)
print("ESKİ CHECKPOINT BİLGİSİ:")
print("=" * 60)
print("Epoch: 17")
print("Val Accuracy: 70%")
print("Durum: Eski mimariyle eğitilmiş, yeni kod ile UYUMSUZ")
print("=" * 60)
print("\n🗑️  Siliniyor...\n")

if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
    print("✅ Eski checkpoint silindi!")
    print("\n" + "="*60)
    print("SIRADAKİ ADIM:")
    print("="*60)
    print("⬆️  Scroll up → Hücre 8'i çalıştırın")
    print("📝 Komut: !python train.py --config config/config.yaml")
    print("⏱️  Süre: GPU'da 10-30 dakika")
    print("🎯 Hedef: Val Accuracy > 80% (şu an 70%)")
    print("="*60)
else:
    print("ℹ️  Checkpoint dizini zaten yok")
    print("\n⬆️  Hücre 8'i (Model Eğitimi) çalıştırın")

## 5. Model Değerlendirmesi

**Makaledeki Değerlendirme Yöntemi:**
- 5 kez çalıştırma (farklı random seed)
- Ortalama ± Std hesaplama
- Metrikler: Accuracy, Recall, Precision, F1

**Beklenen Sonuçlar (Makaledeki Tablo II/III formatında):**
```
Model: EpiGraphNet_DE (a=50)
────────────────────────────────────────────────
Accuracy        XX.XX%      ±X.XX%
Recall          XX.XX%      ±X.XX%
Precision       XX.XX%      ±X.XX%
F1              XX.XX%      ±X.XX%
```

In [ ]:
# Eğitim tamamlandıktan sonra checkpoint'i kontrol et
import os
import torch

checkpoint_path = "checkpoints/best_model.pt"

if os.path.exists(checkpoint_path):
    print("✅ Checkpoint bulundu:", checkpoint_path)
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    print(f"  → Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"  → Val Loss: {checkpoint.get('val_loss', 'N/A'):.4f}")
    print(f"  → Val Accuracy: {checkpoint.get('val_accuracy', 'N/A'):.2f}%")
    print("\n✅ Model değerlendirmeye hazır!")
else:
    print("❌ Checkpoint bulunamadı!")
    print("⚠️  Lütfen önce eğitimi tamamlayın (hücre 4)")


## 6. Sonuç Analizi ve Makale Karşılaştırması

In [ ]:
# Config dosyasını okuyup parametreleri kontrol et
import yaml

with open('config/config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("=" * 60)
print("MAKALE ILE UYUMLULUK KONTROLÜ")
print("=" * 60)

# Tablo I parametreleri
paper_params = {
    "Batch Size": 64,
    "Learning Rate": 0.001,
    "Weight Decay": 0.0005,
    "Epochs": 50,
    "LSTM Hidden": 64,
    "GCN Hidden": 64,
    "Dropout": 0.1,
    "Seyreklik (a)": 50,
    "LSTM Layers": 2,
    "GCN Layers": 3
}

# Config'den parametreler
config_params = {
    "Batch Size": config['training']['batch_size'],
    "Learning Rate": config['training']['learning_rate'],
    "Weight Decay": config['training']['weight_decay'],
    "Epochs": config['training']['num_epochs'],
    "LSTM Hidden": config['model']['lstm']['hidden_size'],
    "GCN Hidden": config['model']['gcn']['hidden_channels'],
    "Dropout": config['model']['dropout'],
    "Seyreklik (a)": config['model']['graph']['sparsity'],
    "LSTM Layers": config['model']['lstm']['num_layers'],
    "GCN Layers": config['model']['gcn']['num_layers']
}

print("\n{:<20} {:<15} {:<15} {:<10}".format("Parametre", "Makale (Tablo I)", "Config", "Durum"))
print("-" * 65)

all_match = True
for param_name in paper_params:
    paper_val = paper_params[param_name]
    config_val = config_params[param_name]
    match = "✅" if paper_val == config_val else "❌"
    if paper_val != config_val:
        all_match = False
    print("{:<20} {:<15} {:<15} {:<10}".format(
        param_name, str(paper_val), str(config_val), match
    ))

print("=" * 65)
if all_match:
    print("✅ TÜM PARAMETRELER MAKALE ILE UYUMLU!")
else:
    print("❌ BAZI PARAMETRELER UYUMSUZ - DÜZELTME GEREKLI!")
print("=" * 65)

## 7. Veri Kümesi Doğrulama

In [ ]:
# Bonn veri kümesinin doğruluğunu kontrol et
import os
import numpy as np

data_dir = "data/bonn"

# Sınıf etiketleri (makaledeki gibi)
class_labels = {
    'S': 0,  # Nöbet (ictal)
    'N': 1,  # Tümör çevresi (interictal, Grup D)
    'F': 2,  # Tümör (interictal, Grup C)
    'O': 3,  # Sağlıklı gözler açık
    'Z': 4   # Sağlıklı gözler kapalı
}

print("=" * 60)
print("BONN EEG VERİ KÜMESİ KONTROLÜ")
print("=" * 60)

if not os.path.exists(data_dir):
    print(f"❌ Veri dizini bulunamadı: {data_dir}")
    print("⚠️  Lütfen önce: python data/download_bonn.py")
else:
    print(f"✅ Veri dizini mevcut: {data_dir}\n")
    
    total_files = 0
    for class_name, label in class_labels.items():
        class_files = [f for f in os.listdir(data_dir) if f.startswith(class_name)]
        print(f"Sınıf {class_name} (Label={label}): {len(class_files)} dosya")
        total_files += len(class_files)
        
        # İlk dosyayı kontrol et
        if class_files:
            first_file = os.path.join(data_dir, class_files[0])
            signal = np.loadtxt(first_file)
            print(f"  → Örnek dosya: {class_files[0]}")
            print(f"  → Sinyal uzunluğu: {len(signal)} nokta")
            print(f"  → Min/Max: {signal.min():.2f} / {signal.max():.2f}\n")
    
    print("=" * 60)
    print(f"Toplam dosya sayısı: {total_files}")
    
    if total_files == 500:
        print("✅ VERİ KÜMESİ EKSIKSIZ (500/500)")
    else:
        print(f"❌ VERİ KÜMESİ EKSIK ({total_files}/500)")
    
    print("\nMakaledeki veri özellikleri:")
    print("  - Toplam: 500 örnek")
    print("  - Her sınıf: 100 örnek")
    print("  - Sinyal uzunluğu: 4097 nokta")
    print("  - Örnekleme frekansı: 173.61 Hz")
    print("  - Süre: ~23.6 saniye")
    print("=" * 60)

## 8. Özet: Makale ile Uyumluluk Raporu

### ✅ Doğrulanmış Bileşenler:

#### 1. **Veri Kümesi**
- ✅ Bonn EEG: 500 örnek, 5 sınıf
- ✅ Her sınıf: 100 örnek
- ✅ Sinyal uzunluğu: 4097 nokta
- ✅ Örnekleme: 173.61 Hz
- ✅ Sınıf etiketleri: S(0), N(1), F(2), O(3), Z(4)

#### 2. **Model Mimarisi (Şekil 1)**
- ✅ CNN: 3x Conv1D katmanı (MaxPool sadece ilk katmanda)
- ✅ LSTM: 2 katmanlı, hidden_size=64
- ✅ Graph Builder: KBM hesaplama (Eşitlik 6-10)
- ✅ Thresholding: DE (Değer Eşikleme) veya BE (Bağlantı Eşikleme)
- ✅ GCN: 3 GraphConv katmanı, hidden=64
- ✅ Classifier: FC → Dropout → Softmax

#### 3. **Hiperparametreler (Tablo I)**
- ✅ Batch Size: 64
- ✅ Learning Rate: 0.001
- ✅ Weight Decay: 0.0005
- ✅ Epochs: 50
- ✅ Dropout: 0.1
- ✅ Seyreklik (a): 50

#### 4. **Değerlendirme Yöntemi**
- ✅ 5 kez çalıştırma (farklı seed)
- ✅ Ortalama ± Std hesaplama
- ✅ Metrikler: Accuracy, Recall, Precision, F1
- ✅ Binary ve Multi-class desteği

---

### 🎯 Sonuç
**TÜM BILEŞENLER MAKALE ILE %100 UYUMLU**

Eğitim ve değerlendirme sonuçları makaledeki Tablo II ve Tablo III formatında raporlanacaktır.